In [19]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
# 📊 FUNCIÓN LEER_HOJA ACTUALIZADA
# Funciona tanto localmente (con .env) como en GitHub Actions (con secrets)

import os
import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from dotenv import load_dotenv
import json

def configurar_google_sheets():
    """
    Configura la conexión a Google Sheets
    Funciona tanto localmente (con .env) como en GitHub Actions (con secrets)
    """
    # Cargar variables de entorno
    load_dotenv()
    
    # Configuración
    SHEET_ID = os.getenv('GOOGLE_SHEET_ID')
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    
    # Método 1: Intentar usar archivo JSON local (para ejecución local)
    SERVICE_ACCOUNT_FILE = os.getenv('GOOGLE_SERVICE_ACCOUNT_FILE')
    if SERVICE_ACCOUNT_FILE and os.path.exists(SERVICE_ACCOUNT_FILE):
        print("🔑 Usando archivo JSON local para autenticación")
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    else:
        # Método 2: Usar JSON desde variable de entorno (para GitHub Actions)
        SERVICE_ACCOUNT_JSON = os.getenv('GOOGLE_SERVICE_ACCOUNT_JSON')
        if SERVICE_ACCOUNT_JSON:
            print("🔑 Usando JSON desde variable de entorno para autenticación")
            # Crear credenciales desde el JSON
            service_account_info = json.loads(SERVICE_ACCOUNT_JSON)
            creds = Credentials.from_service_account_info(service_account_info, scopes=SCOPES)
        else:
            raise ValueError("❌ No se encontraron credenciales de Google. Configura GOOGLE_SERVICE_ACCOUNT_FILE o GOOGLE_SERVICE_ACCOUNT_JSON")
    
    # Crear servicio
    service = build('sheets', 'v4', credentials=creds)
    
    return service, SHEET_ID

# Configurar conexión global
service, SHEET_ID = configurar_google_sheets()
print(f"✅ Conectado a Google Sheet: {SHEET_ID}")

def leer_hoja(nombre_hoja='REGISTRO_DIARIO'):
    """
    Lee TODA la hoja completa desde Google Sheets, sin importar cuántas filas tenga.
    Se adapta automáticamente al crecimiento de datos y maneja celdas vacías.
    
    Args:
        nombre_hoja (str): Nombre de la hoja a leer
        
    Returns:
        pandas.DataFrame: DataFrame con los datos de la hoja o None si hay error
    """
    try:
        # Leer toda la hoja (sin especificar rango)
        range_name = f'{nombre_hoja}'
        
        result = service.spreadsheets().values().get(
            spreadsheetId=SHEET_ID, 
            range=range_name
        ).execute()
        
        values = result.get('values', [])
        
        if values:
            if len(values) > 1:
                # Obtener encabezados y número de columnas esperadas
                headers = values[0]
                num_columns = len(headers)
                
                # Normalizar todas las filas para que tengan el mismo número de columnas
                data_rows = []
                for row in values[1:]:
                    # Si la fila tiene menos columnas, completar con cadenas vacías
                    if len(row) < num_columns:
                        row_completa = row + [''] * (num_columns - len(row))
                        data_rows.append(row_completa)
                    else:
                        data_rows.append(row)
                
                df = pd.DataFrame(data_rows, columns=headers)
            else:
                df = pd.DataFrame(values)
            
            print(f"✅ Se leyeron {len(df)} filas de {nombre_hoja}")
            print(f"📊 Columnas: {list(df.columns)}")
            return df
        else:
            print("❌ No se encontraron datos")
            return None
            
    except Exception as e:
        print(f"❌ Error al leer la hoja {nombre_hoja}: {e}")
        return None

print("🎯 Función leer_hoja() lista para usar")

🔑 Usando archivo JSON local para autenticación
✅ Conectado a Google Sheet: 14AtL1_MHWaN1JR_dbbddif9w0ujlx4wFrEuelmv6gfs
🎯 Función leer_hoja() lista para usar


In [46]:
df3=leer_hoja("REGISTRO_CALENDARIO")
df4=leer_hoja("PAGOS")
df5=leer_hoja("PAGOSCHECK")

✅ Se leyeron 704 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime', 'ID_Registro']
✅ Se leyeron 21 filas de PAGOS
📊 Columnas: ['id_pago', 'DNI', 'Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'horas_normales', 'horas_extra', 'horas_extra_no_pagadas', 'pago_base', 'monto_total', 'horas_deberia']
✅ Se leyeron 21 filas de PAGOS
📊 Columnas: ['id_pago', 'DNI', 'Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'horas_normales', 'horas_extra', 'horas_extra_no_pagadas', 'pago_base', 'monto_total', 'horas_deberia']
✅ Se leyeron 41 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']
✅ Se leyeron 41 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']


In [22]:
print(df5)

        Colaborador periodo_inicio periodo_fin  fecha_pago  check
0      Andrea Yuqui     16/06/2025  15/07/2025  16/07/2025  Listo
1      Andrea Yuqui     16/07/2025  15/08/2025  16/08/2025  Listo
2      Andrea Yuqui     16/08/2025  15/09/2025  16/09/2025       
3          Benjamin     02/05/2025  01/06/2025  02/06/2025  Listo
4          Benjamin     02/06/2025  01/07/2025  02/07/2025  Listo
5          Benjamin     02/07/2025  01/08/2025  02/08/2025  Listo
6          Benjamin     02/08/2025  01/09/2025  04/09/2025       
7      Edison Yuqui     16/06/2025  15/07/2025  16/07/2025  Listo
8      Edison Yuqui     16/07/2025  15/08/2025  16/08/2025  Listo
9      Edison Yuqui     16/08/2025  15/09/2025  16/09/2025       
10         Fiorella     02/06/2025  01/07/2025  02/07/2025  Listo
11         Fiorella     02/07/2025  01/08/2025  02/08/2025  Listo
12             Gary     02/06/2025  01/07/2025  02/07/2025  Listo
13             Gary     02/07/2025  01/08/2025  02/08/2025  Listo
14        

In [23]:
import pandasql as psql
from datetime import datetime
query2="""Select Colaborador,periodo_inicio,periodo_fin, fecha_pago
from df4
Group By Colaborador,fecha_pago
"""
periodos=psql.sqldf(query2,locals())
periodos["check"]=""
periodos.loc[0,"fecha_pago"]
filascount=len(periodos) 
hoy=datetime.today()
for i in range(filascount):
    fecha=periodos.loc[i,"fecha_pago"]
    fecha=datetime.strptime(fecha, "%d/%m/%Y")
    if fecha<hoy:
        periodos.loc[i,"check"]="Listo"

print(periodos)
df5=periodos


        Colaborador periodo_inicio periodo_fin  fecha_pago  check
0      Andrea Yuqui     16/06/2025  15/07/2025  16/07/2025  Listo
1      Andrea Yuqui     16/07/2025  15/08/2025  16/08/2025  Listo
2      Andrea Yuqui     16/08/2025  15/09/2025  16/09/2025       
3          Benjamin     02/05/2025  01/06/2025  02/06/2025  Listo
4          Benjamin     02/06/2025  01/07/2025  02/07/2025  Listo
5          Benjamin     02/07/2025  01/08/2025  02/08/2025  Listo
6          Benjamin     02/08/2025  01/09/2025  02/09/2025  Listo
7      Edison Yuqui     16/06/2025  15/07/2025  16/07/2025  Listo
8      Edison Yuqui     16/07/2025  15/08/2025  16/08/2025  Listo
9      Edison Yuqui     16/08/2025  15/09/2025  16/09/2025       
10         Fiorella     02/06/2025  01/07/2025  02/07/2025  Listo
11         Fiorella     02/07/2025  01/08/2025  02/08/2025  Listo
12             Gary     02/06/2025  01/07/2025  02/07/2025  Listo
13             Gary     02/07/2025  01/08/2025  02/08/2025  Listo
14        

In [24]:
import pandasql as psql
query="""Select Colaborador
from df3
group by Colaborador
"""

result=psql.sqldf(query,locals())
colaboradores=result["Colaborador"].tolist()
print(colaboradores[0])

for colaborador in colaboradores:
    query=f"""Select *
    from df3
    where Colaborador='{colaborador}'
    """
    result=psql.sqldf(query,locals())
    #result.to_excel(f"REGISTRO_CALENDARIO_{colaborador}.xlsx")
    print(result)

Andrea Yuqui
     Colaborador HoraEntrada    HoraSalida FechaEntrada FechaSalida Minutos  \
0   Andrea Yuqui    17:38:34       0:03:04    11/7/2025   12/7/2025     385   
1   Andrea Yuqui    16:00:33      20:02:18    12/7/2025   12/7/2025     242   
2   Andrea Yuqui     8:01:19  00:03:19.000    13/7/2025   14/7/2025     962   
3   Andrea Yuqui    15:59:51       0:07:52    14/7/2025   15/7/2025     488   
4   Andrea Yuqui    16:00:51       0:08:00    15/7/2025   16/7/2025     487   
5   Andrea Yuqui    16:01:03       0:02:13    16/7/2025   17/7/2025     481   
6   Andrea Yuqui    16:00:10       0:13:19    17/7/2025   18/7/2025     493   
7   Andrea Yuqui    16:03:07       0:02:40    18/7/2025   19/7/2025     480   
8   Andrea Yuqui    16:21:06       0:01:18    19/7/2025   20/7/2025     460   
9   Andrea Yuqui     7:01:36       0:02:40    20/7/2025   21/7/2025    1021   
10  Andrea Yuqui    16:00:56       0:32:59    21/7/2025   22/7/2025     512   
11  Andrea Yuqui    16:00:37       0:03

In [25]:
type(result.loc[0,'FechaEntrada'])
result=result.values.tolist()

In [26]:

for colaborador in colaboradores: 
    for periodo in periodos:
        if colaborador == "Andrea Yuqui" and periodo[0]=="Andrea Yuqui":
            
            query=f"""Select *
                from df3
                where Colaborador='{colaborador}'
            """
            result=psql.sqldf(query,locals())

            result
            print(f"Colaborador {colaborador} encontrado en el periodo {periodo[1]} - {periodo[2]}.")
print(result)


[['Valentino', '19:42:56', '19:43:18', '2/6/2025', '2/6/2025', '0', '0', '0', '1', '', '', '', 'No'], ['Valentino', '8:59:32', '9:24:59', '5/6/2025', '5/6/2025', '25', '0', '25', '9', '', 'Comparacion de los stocks de peri collection y investigacion de los pasos para que el stock de mariana actualice correctamente el estado de los pedidos', '', 'No'], ['Valentino', '18:25:43', '18:31:27', '5/6/2025', '5/6/2025', '6', '0', '6', '10', '', 'Soporte a andre', '', 'No'], ['Valentino', '11:12:13', '11:13:11', '6/6/2025', '6/6/2025', '1', '0', '1', '11', '', 'Me olvide marcar desde las 9:50. Hice soporte a Andrea y avancé con lo de añadir las nuevas tallas a peri collection', '', 'No'], ['Valentino', '13:28:08', '13:29:03', '7/6/2025', '7/6/2025', '1', '0', '1', '13', '', '', '', 'No'], ['Valentino', '15:05:58', '15:51:04', '7/6/2025', '7/6/2025', '45', '0', '45', '19', '', 'Avance del filtro para QRs para Mariana', '', 'No'], ['Valentino', '15:59:45', '18:14:01', '7/6/2025', '7/6/2025', '134

In [27]:
# 📦 INSTALAR LIBRERÍAS
%pip install pandasql openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
df4

id_pago       DNI      Colaborador periodo_inicio periodo_fin  fecha_pago  \
0        1        29  Mariana Benites     08/08/2025  07/09/2025  08/09/2025   
1        2        30     Renata Rojas     28/07/2025  27/08/2025  28/08/2025   
2        3        31    Keiner NIeves     05/08/2025  19/08/2025  20/08/2025   
3        4        31    Keiner NIeves     20/08/2025  04/09/2025  05/09/2025   
4        5        32   Stephany Perez     05/08/2025  19/08/2025  20/08/2025   
5        6        32   Stephany Perez     20/08/2025  04/09/2025  05/09/2025   
6        7        33     Andrea Yuqui     16/06/2025  15/07/2025  16/07/2025   
7        8        33     Andrea Yuqui     16/07/2025  15/08/2025  16/08/2025   
8        9        33     Andrea Yuqui     16/08/2025  15/09/2025  16/09/2025   
9       10        34     Edison Yuqui     16/06/2025  15/07/2025  16/07/2025   
10      11        34     Edison Yuqui     16/07/2025  15/08/2025  16/08/2025   
11      12        34     Edison Yuqui     16/08/2025  15/09/2025  16/09/2025   
12      13        35       Rosa Clavo     16/06/2025  15/07/2025  16/07/2025   
13      14        35       Rosa Clavo     16/07/2025  15/08/2025  16/08/2025   
14      15        35       Rosa Clavo     16/08/2025  15/09/2025  16/09/2025   
15      16        36     Lucero Ocaña     16/06/2025  15/07/2025  16/07/2025   
16      17        36     Lucero Ocaña     16/07/2025  15/08/2025  16/08/2025   
17      18        36     Lucero Ocaña     16/08/2025  15/09/2025  16/09/2025   
18      19        37    Paulina Cunia     16/06/2025  15/07/2025  16/07/2025   
19      20        37    Paulina Cunia     16/07/2025  15/08/2025  16/08/2025   
20      21        37    Paulina Cunia     16/08/2025  15/09/2025  16/09/2025   
21      22  74754286         Benjamin     02/05/2025  01/06/2025  02/06/2025   
22      23  74754286         Benjamin     02/06/2025  01/07/2025  02/07/2025   
23      24  74754286         Benjamin     02/07/2025  01/08/2025  02/08/2025   
24      25  74754286         Benjamin     02/08/2025  01/09/2025  02/09/2025   
25      26        39        Valentino     02/05/2025  01/06/2025  02/06/2025   
26      27        39        Valentino     02/06/2025  01/07/2025  02/07/2025   
27      28        39        Valentino     02/07/2025  01/08/2025  02/08/2025   
28      29        40             Gary     02/06/2025  01/07/2025  02/07/2025   
29      30        40             Gary     02/07/2025  01/08/2025  02/08/2025   
30      31        40             Gary     02/08/2025  01/09/2025  02/09/2025   
31      32        41         Fiorella     02/06/2025  01/07/2025  02/07/2025   
32      33        41         Fiorella     02/07/2025  01/08/2025  02/08/2025   
33      34        42             Luis     02/07/2025  01/08/2025  02/08/2025   
34      35        45  Patricia Sullca     02/07/2025  01/08/2025  02/08/2025   
35      36        45  Patricia Sullca     02/08/2025  01/09/2025  02/09/2025   

   horas_normales horas_extra horas_extra_no_pagadas pago_base monto_total  \
0           102,3           0                  14,93      1100        1100   
1            37,3           0                  10,77       700         700   
2           35,85           0                   4,63      1200        1200   
3           71,58           0                   5,72      1200        1200   
4           52,88           0                   0,63      1000        1000   
5           87,03           0                   1,27      1000        1000   
6           42,32           0                   0,42         0       46,55   
7          276,68           0                   12,6         0      304,35   
8          165,08           0                   2,43         0      181,59   
9           35,72           0                   3,45         0       37,86   
10         176,45           0                  12,67         0      187,04   
11          113,8           0                  18,15         0      120,63   
12           6,18           

In [47]:
periodos=leer_hoja("PAGOSCHECK")

✅ Se leyeron 41 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']


In [30]:
df3.head()

Colaborador HoraEntrada HoraSalida FechaEntrada FechaSalida Minutos  \
0   Valentino    19:42:56   19:43:18     2/6/2025    2/6/2025       0   
1    Benjamin    18:58:53   19:45:02     2/6/2025    2/6/2025      46   
2    Benjamin    14:49:23   15:04:42     3/6/2025    3/6/2025      15   
3    Fiorella    14:54:17   15:04:49     3/6/2025    3/6/2025      11   
4    Benjamin    15:38:15   15:40:44     3/6/2025    3/6/2025       2   

  Minutos_extras Minutos_normales ID_Calendario Alerta Descripcion  \
0              0                0             1                      
1              0               46             2                      
2              0               15             3                      
3              0               11             4                      
4              0                2             5                      

  Observacion Extratime  
0                    No  
1                    No  
2                    No  
3                    No  
4                    No

In [31]:
filas=[0,3]
periodos.iloc[filas]
print(periodos.iloc[filas])

    Colaborador periodo_inicio periodo_fin  fecha_pago  check
0  Andrea Yuqui     16/06/2025  15/07/2025  16/07/2025  Listo
3      Benjamin     02/05/2025  01/06/2025  02/06/2025  Listo


In [6]:
# 🚀 EJECUTAR PROCESAMIENTO DE REPORTES PARA HOY
procesar_reportes_hoy()

🚀 === PROCESANDO REPORTES PARA HOY ===
📅 Fecha actual: 04/09/2025

📊 Cargando datos de Google Sheets...
✅ Se leyeron 36 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']
🔄 Rango automático: A1:E37
✅ Se leyeron 36 filas de PAGOSCHECK
📊 Columnas: ['Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'check']
🔄 Rango automático: A1:E37
✅ Se leyeron 710 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime']
🔄 Rango automático: A1:M711
✅ Se leyeron 710 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime']
🔄 Rango automático: A1:M711
✅ Se leyeron 19 filas de VENDEDORAS
📊 Co

True

# 📧 CONFIGURACIÓN REQUERIDA PARA GMAIL

Para que funcione el envío de correos, necesitas configurar las siguientes variables en tu archivo `.env`:

```env
# Configuración Gmail
GMAIL_USER=tu-email@gmail.com
GMAIL_APP_PASSWORD=tu-contraseña-de-aplicacion
```

## 🔑 ¿Cómo obtener la contraseña de aplicación de Gmail?

1. **Ir a tu cuenta de Google** → Configuración de seguridad
2. **Activar la verificación en 2 pasos** (si no la tienes)
3. **Generar una contraseña de aplicación**:
   - Ve a "Contraseñas de aplicaciones"
   - Selecciona "Correo" y "Otro"
   - Copia la contraseña de 16 caracteres generada
4. **Agregar al archivo .env** sin espacios ni guiones

⚠️ **IMPORTANTE**: Usa la contraseña de aplicación, NO tu contraseña normal de Gmail.

# REGISTRO CALENDARIO

In [48]:
Registro_calendario=leer_hoja("REGISTRO_CALENDARIO")

✅ Se leyeron 704 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime', 'ID_Registro']


In [26]:
Registro_calendario.tail()

Colaborador HoraEntrada HoraSalida FechaEntrada FechaSalida Minutos  \
763  Lucero Ocaña     7:01:25    8:36:22     7/9/2025    7/9/2025      95   
764    Rosa Clavo     9:15:10   10:12:34     7/9/2025    7/9/2025      57   
765  Lucero Ocaña     9:18:27   11:21:05     7/9/2025    7/9/2025     123   
766  Andrea Yuqui     8:01:48                7/9/2025                   0   
767  Edison Yuqui     8:01:25                7/9/2025                   0   

    Minutos_extras Minutos_normales ID_Calendario Alerta Descripcion  \
763              0               95           764                      
764              0               57           765                      
765              0              123           766                      
766              0                0           767     ⚠️               
767              0                0           768     ⚠️               

    Observacion Extratime ID_Registro  
763                    No    30778371  
764                    No    a917a137  
765                    No    743ce7ac  
766                    No    d6b7e751  
767                    No    4a4616b4

In [51]:
Registro_diario.tail()

ID     Fecha      Hora    Etapa      Colaborador Día         Mes  \
1569  8b7ebb49  8/9/2025  13:00:14  Entrada     Lucero Ocaña   8  septiembre   
1570  83ee89e3  8/9/2025  13:00:38   Salida  Patricia Sullca   8  septiembre   
1571  472cc191  8/9/2025  13:31:29   Salida   Stephany Perez   8  septiembre   
1572  c1704c5b  8/9/2025  14:00:39  Entrada  Patricia Sullca   8  septiembre   
1573  9cba7631  8/9/2025  14:02:15   Salida       Rosa Clavo   8  septiembre   

       Año      PERIODO                               Rol Descripcion  \
1569  2025  1raQuincena                    Administrativo               
1570  2025  1raQuincena  Asesor Comercial Peri Collection               
1571  2025  1raQuincena                    Peri Institute               
1572  2025  1raQuincena  Asesor Comercial Peri Collection               
1573  2025  1raQuincena                    Ropa Deportiva               

     Captura de petición de horas extra Observacion  
1569                                                 
1570                                                 
1571                                                 
1572                                                 
1573

In [50]:
FilasCalendario=len(Registro_calendario)
Registro_calendario.loc[FilasCalendario-1,"Colaborador"]

'Rosa Clavo'

In [49]:
Registro_diario=leer_hoja("REGISTRO_DIARIO")

✅ Se leyeron 1574 filas de REGISTRO_DIARIO
📊 Columnas: ['ID', 'Fecha', 'Hora', 'Etapa', 'Colaborador', 'Día', 'Mes', 'Año', 'PERIODO', 'Rol', 'Descripcion', 'Captura de petición de horas extra', 'Observacion']


In [52]:

filasDiarios=len(Registro_diario)

for i in range(FilasCalendario):
    if Registro_calendario.loc[FilasCalendario -1 -i,"HoraSalida"]=="":
        Colaborador=Registro_calendario.loc[FilasCalendario -1 -i,"Colaborador"] 
        ID_Registro=Registro_calendario.loc[FilasCalendario -1 -i,"ID_Registro"] 
        for j in range(filasDiarios):
            if Registro_diario.loc[filasDiarios -1 -j,"ID"]==ID_Registro:
                print(Registro_diario.loc[filasDiarios -1 -j,"Colaborador"])
                print(Registro_diario.loc[filasDiarios -1 -j,"Etapa"])
                index=filasDiarios -1 -j
                for z in range (index, filasDiarios):
                    if Registro_diario.loc[z,"Etapa"]=="Salida" and Registro_diario.loc[z,"Colaborador"]==Colaborador:
                        print(Registro_diario.loc[z,"Hora"])
                        break
                    else:
                        print("No hay salida registrada")

In [85]:
# buscar nuevos registros
# Usar iloc para evitar KeyError por índices negativos
ultimoID = Registro_calendario.loc[FilasCalendario - 1, "ID_Registro"]
IndexNuevo = 0
for i in range(filasDiarios):
    if ultimoID == Registro_diario.loc[filasDiarios - 1 - i, "ID"]:
        IndexNuevo = filasDiarios - 1 - i
        break
IndexNuevo=IndexNuevo+1
Entradas=[]
i=0
for j in range(IndexNuevo, filasDiarios):
    if Registro_diario.loc[j, "Etapa"] == "Entrada":
        entrada={
            "Colaborador": Registro_diario.loc[j, "Colaborador"],
            "HoraEntrada": Registro_diario.loc[j, "Hora"],
            "FechaEntrada": Registro_diario.loc[j, "Fecha"],
            "ID_Registro": Registro_diario.loc[j, "ID"]
        }
        Entradas.append(entrada)


Entradas[0]["Colaborador"]

'Andrea Yuqui'

In [88]:
# 📝 FUNCIÓN SIMPLE PARA ESCRIBIR A GOOGLE SHEETS
def escribir_a_sheets(nombre_hoja, datos):
    """
    Escribe datos directamente a Google Sheets al final de la hoja
    
    Args:
        nombre_hoja (str): Nombre de la hoja
        datos (list): Lista de listas con los datos
    """
    try:
        # Obtener última fila
        result = service.spreadsheets().values().get(
            spreadsheetId=SHEET_ID,
            range=f'{nombre_hoja}!A:A'
        ).execute()
        
        ultima_fila = len(result.get('values', [])) + 1
        
        # Escribir datos
        service.spreadsheets().values().update(
            spreadsheetId=SHEET_ID,
            range=f'{nombre_hoja}!A{ultima_fila}',
            valueInputOption='USER_ENTERED',
            body={'values': datos}
        ).execute()
        
        print(f"✅ {len(datos)} filas escritas en {nombre_hoja}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

# Convertir tu array Entradas y escribir
datos_para_escribir = []
for entrada in Entradas:
    fila = [
        entrada["Colaborador"],
        entrada["HoraEntrada"],
        "",
        entrada["FechaEntrada"], 
        "",
        "",  # HoraSalida vacía
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        entrada["ID_Registro"]
    ]
    datos_para_escribir.append(fila)

# Escribir a la hoja
escribir_a_sheets("REGISTRO_CALENDARIO", datos_para_escribir)

✅ 70 filas escritas en REGISTRO_CALENDARIO


In [ ]:

for j in range(filasDiarios):
    if Registro_diario.loc[filasDiarios -1 -j,"ID"]=="4a4616b4":
        print(Registro_diario.loc[filasDiarios -1 -j,"Colaborador"])
        print(Registro_diario.loc[filasDiarios -1 -j,"Etapa"])
        index=filasDiarios -1 -j
        for z in range (index, filasDiarios):
            if Registro_diario.loc[z,"Etapa"]=="Salida" and Registro_diario.loc[z,"Colaborador"]=="Edison Yuqui":
                print(Registro_diario.loc[z,"Hora"])
                break
            else:
                print("No hay salida registrada")

✅ Se leyeron 1555 filas de REGISTRO_DIARIO
📊 Columnas: ['ID', 'Fecha', 'Hora', 'Etapa', 'Colaborador', 'Día', 'Mes', 'Año', 'PERIODO', 'Rol', 'Descripcion', 'Captura de petición de horas extra', 'Observacion']
Edison Yuqui
Entrada
No hay salida registrada
No hay salida registrada
8:36:22
